<a href="https://colab.research.google.com/github/JonathanSum/Hugging-Face-Course/blob/main/Chapter_2_Behind_the_pipeline_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 542 kB 10.2 MB/s 
     |████████████████████████████████| 2.6 MB 36.4 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
     |████████████████████████████████| 243 kB 45.1 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 118 kB 52.2 MB/s 
     |████████████████████████████████| 3.3 MB 32.6 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 636 kB 34.0 MB/s 
     |████████████████████████████████| 1.1 MB 44.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [27]:
raw_inputs = [
    "Happy Sugar Life is all above love", 
    "Love is not about making someone to live for yourself.",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 3407, 5699, 2166, 2003, 2035, 2682, 2293,  102,    0,    0,    0,
            0],
        [ 101, 2293, 2003, 2025, 2055, 2437, 2619, 2000, 2444, 2005, 4426, 1012,
          102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [28]:
inputs[0]   #It is 13 here because it use the max length of the sentence, which is the second to do padding for inference. I guess.

Encoding(num_tokens=13, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [29]:
inputs[1]

Encoding(num_tokens=13, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [30]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier([
    "Happy Sugar Life is all above love", 
    "Love is not about making someone to live for yourself.",
])

[{'label': 'POSITIVE', 'score': 0.9996786117553711},
 {'label': 'POSITIVE', 'score': 0.9912222623825073}]

In [31]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
import torch

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)


torch.Size([2, 13, 768])


In [36]:
outputs["last_hidden_state"].shape

torch.Size([2, 13, 768])

In [37]:
outputs[0].shape

torch.Size([2, 13, 768])

In [38]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [39]:
print(outputs.logits.shape)

torch.Size([2, 2])


In [40]:
print(outputs.logits)

tensor([[-3.8937,  4.1488],
        [-2.3147,  2.4120]], grad_fn=<AddmmBackward>)


In [41]:
outputs

SequenceClassifierOutput([('logits', tensor([[-3.8937,  4.1488],
                                   [-2.3147,  2.4120]], grad_fn=<AddmmBackward>))])

In [42]:
model.config.id2label


{0: 'NEGATIVE', 1: 'POSITIVE'}